# 데이터 로드 및 정제

In [ ]:
!pip install transformers

In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import urllib.request
from sklearn import preprocessing
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint  #라이브러리의 소스 코드를 열어보자!!!!

In [ ]:
df = pd.read_excel('/content/식품 Q&A 데이터.xlsx')
df.head()

,발화문 (Q),발화문 (A),QA번호,인텐트 (대),인텐트 (중),인텐트 (소)
0,컵밥 2만원 이상을 사면 무료 배송으로 아는데 여기가 죽도 추가로 몇 개 더 주문해...,박스에 합포가 되지 않을 수도 있어 정확한 답변 드리지 못하는 점 양해 부탁드립니다.,46175,배송,비용,질문
1,해마로 기준 6개 이상이고 볶음밥 기준으로는 50개 이상인데 해마로 케이준 1봉과 ...,일단 두 제품이 같이 담기면 최소 수량 부과인 6개 이상은 시스템 상 자동부과가 됩니다.,40543,배송,비용,질문
2,마트 원데이는 무료 배송이라고 되있는데 왜 결제를 하려고 하니까 배송비 3000원이...,1KG 5 팩 이 상 무료배송 상품입니다.,15426,배송,비용,질문
3,제주도 추가 배송료 잇나요?,문의해주셔서 감사해요.,226944,배송,비용,질문
4,물건이 오지도 않았는데 취소하게 되면 3천원이 빠지더라구 요 반품 택배사원이 오는 ...,운송장 발행되어 택배사에서 발송하면 반품 배송비가 발생해요.,229560,배송,비용,질문


In [ ]:
data = df.loc[:, ['발화문 (Q)', '발화문 (A)', '인텐트 (대)']]
data

,발화문 (Q),발화문 (A),인텐트 (대)
0,컵밥 2만원 이상을 사면 무료 배송으로 아는데 여기가 죽도 추가로 몇 개 더 주문해...,박스에 합포가 되지 않을 수도 있어 정확한 답변 드리지 못하는 점 양해 부탁드립니다.,배송
1,해마로 기준 6개 이상이고 볶음밥 기준으로는 50개 이상인데 해마로 케이준 1봉과 ...,일단 두 제품이 같이 담기면 최소 수량 부과인 6개 이상은 시스템 상 자동부과가 됩니다.,배송
2,마트 원데이는 무료 배송이라고 되있는데 왜 결제를 하려고 하니까 배송비 3000원이...,1KG 5 팩 이 상 무료배송 상품입니다.,배송
3,제주도 추가 배송료 잇나요?,문의해주셔서 감사해요.,배송
4,물건이 오지도 않았는데 취소하게 되면 3천원이 빠지더라구 요 반품 택배사원이 오는 ...,운송장 발행되어 택배사에서 발송하면 반품 배송비가 발생해요.,배송
...,...,...,...
217151,오늘 주문 햇는데요 주말엔 집을 비워서 그러는데 월요일쯤 배송 오게끔 해주실 수잇나요?,주문하신 제품은 월요일 출고 예정이며 화 수요일 정도 수령이 예상됩니다. 업체 당일...,배송
217152,택배 하루만 늦게 오면 안 되나요?,고객님이 미발송 처리된 건이라 처리를 도와드릴 방법이 없네요.,배송
217153,지금 주문하고 9일 토요일 배송받고 싶은데 가능할까요?,지금 주문주신다면 내일 출고되어 토요일에 받아보실 수 있으세요. 참고 하시어 이용에...,배송
217154,지금 주문했는데요 내일 꼭 받아야 합니다 가능한지요?,주문건은 오늘 발송됩니다. 송장번호 조회는 저녁 6시30분 이후부터 가능하세요.,배송


In [ ]:
from sklearn.model_selection import train_test_split

# 데이터 섞기
df_shuffled = data.sample(frac=1, random_state=42)  # random_state는 재현성을 위한 시드값으로 적절한 값을 선택합니다.

# 데이터 나누기 (예: 60% 훈련, 20% 검증, 20% 테스트)
train_data, temp_data = train_test_split(df_shuffled, test_size=0.4, random_state=42)
valid_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# 결과 확인
print(f"훈련 데이터 크기: {len(train_data)}")
print(f"검증 데이터 크기: {len(valid_data)}")
print(f"테스트 데이터 크기: {len(test_data)}")

훈련 데이터 크기: 130293
검증 데이터 크기: 43431
테스트 데이터 크기: 43432


In [ ]:
train_data

,발화문 (Q),발화문 (A),인텐트 (대)
166286,캔 윗부분이 볼록하게 찌그러져 왔는데 어떻게 하죠?,문의 주 신상품은 전체 맞교환 진행 예정이며 수거와 출고가 따로 진행되기 때문에 수...,제품
89526,오늘 주문하면.월요일 날 오나요?,구매하신 제품 꼼꼼히 검수하여요청하신 날짜에 발송하도록 하겠습니다.,배송
46066,언제 오나요?,실수로 토요일에 발송되지 않은 듯합니다.,배송
156934,포도 청포도 유통기한이 어떻게 되나요?,포도 : 21.2.28 청포도 : 21.1.13 감사합니다.,제품
36364,꼭 이번 주에 받아야 하는데 언제 출고되나요?,주문건 확인해보니 금일 출고되어 내일 수령하실 예정입니다 믿고 주문주 신상품인 만큼...,배송
...,...,...,...
44197,언제 배송되나요?,즐거운 오후되시길 소망해요.,배송
118714,오래 먹으려면 냉동에 넣어야 되나요?,문의 해주셔서 감사합니다 우선 저희 ooo상품은 방부제가 첨가되지 않은 신선식품이랍니다.,제품
5183,제주도인데 혹시 상해서 오지는 않겠죠?,제주도는 하루 만에 도착을 하지 않아서 하절기에는 발송하지 않아요 죄송해요.,배송
8956,30일 치먹어보고 만족해서 이번엔 제꺼랑 남자친구 선물용으로도 사려 구함니다. 2번...,현재 주문 물량이 많아 쇼핑백 증정이 어려울 수 있어요. .,포장


In [ ]:
valid_data

,발화문 (Q),발화문 (A),인텐트 (대)
41136,배송이 언제 되나요?,주문하신 상품은 현재 출고되어 배송 중으로 확인됩니다. 자세한 사항은 송장을 통해 ...,배송
193920,순수가 온 30상자에 손잡이 있나요?,네 박스에 별도로 손잡이가 있습니다.,포장
30540,20 프로 쿠폰이 사라진 걸까요?,사은품은 구매 수량에 따라 달라집니다.,행사
42764,싱싱하고 큰 걸로 잘 부탁드립니다 내일 받을 수는 거죠?,순차 출고되어 3/3까지 전량 출고 예정입니다.,배송
105110,독일 자 노탁트비오라브 맥주 효모비오헤페 10월 7일 톡 답변에는 한 개 누락된 제...,이미 발송되었으며 조금만 기다려주시면 감사하겠습니다.,배송
...,...,...,...
192545,생강 2키로 대추 800G 합포는 안 되나요?,신경 써서 출고해 드리도록 하겠습니다. 각각 다른 출고지에서나가는 것이기 때문에 합...,포장
31659,현미 20KG 주문 결제했어요. 찰보리 사은품주시는 거 맞죠?,담당자 분께서 거름망을 빠뜨리신 거 같습니다. 거름망 1개 재발송하겠습니다.,행사
171739,말토덱스트린 함유되 있는데 어느 정도 수준인가요?,상세 페이지에 제품별 함량이 적혀 있어요.,제품
99312,9월 11일 주문했습니다. 언제 발송하는지요?,주문에 대한 출고기한은 9/16까지이며 출고기한 내 순차 출고 드리고 있습니다.,배송


In [ ]:
test_data

,발화문 (Q),발화문 (A),인텐트 (대)
93441,24일 날 주문했는데 아직 집하중인데 무슨 문제 생긴 건가요?,오늘 오후 1시 이전 주문해 주시면 내일 한진택배로 수령하십니다.,배송
44751,언제 오는 건가요?,순차 배송도와드리고 있으며 최대한 빠른 배송될 수 있도록 기사님께 전달도와 드릴게요.,배송
64480,혹시 내일 받을 수 있는지요?,주문하신 제품은 어제 정상적으로 발송되었어요.,배송
26063,초등학생 저학년도 먹어도 되나요?,아보카도 오일은 발연점이 가장 높은 오일로 모든요리에 좋지만 특히 튀김요리에 진가를...,제품
163557,닭가슴살 소세지 어떤 맛이 인기 많나요?,네 돈육소시지와 식감이 동일한 프랑크 소시지가 가장 인기가 많은 제품이십니다.,제품
...,...,...,...
40861,오늘 발송될까요?,27일 발송해 드렸습니다.,배송
212329,주소지 같으면 자동 묶음 배송되나요?,정말 정성껏 만든 막창이니 받아서 맛있게 드셨으면 좋겠구요.,배송
101691,도대체 배송이 왜 이렇게 늦나요?,불편함을 드려 죄송합니다. 현재 KG로지스택배사가 통합 문제로 인해 배송이 지연되고...,배송
89573,혹시 언제 도착하는지 알 수 있을까요?,주문건 당일 출고는 하였는데요택배 측 배송 물량과다로 배송이 지연되고 있는 점 사과...,배송


In [ ]:
def drop_na_and_duplciates(df):
  df = df.dropna()
  df = df.drop_duplicates()
  df = df.reset_index(drop=True)
  return df

#결측값 및 중복 샘플 제거
train_data = drop_na_and_duplciates(train_data)
val_data = drop_na_and_duplciates(valid_data)
test_data = drop_na_and_duplciates(test_data)

print('훈련용 샘플 개수 :',len(train_data))
print('검증용 샘플 개수 :',len(val_data))
print('테스트용 샘플 개수 :',len(test_data))

훈 련 용 샘 플 개 수 : 130276
검 증 용 샘 플 개 수 : 43429
테 스 트 용 샘 플 개 수 : 43429


# BERT 의 입력

In [ ]:
#토큰화를 위해 BERT 토크나이저를 로드
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

#훈련 데이터의 첫번째 샘플을 선택하여 토크나이저의 전처리 결과

max_seq_len = 128  #최대 길이: 128

sent1 = train_data['발화문 (Q)'].iloc[0]
sent2 = train_data['발화문 (A)'].iloc[0]

print('문장1 :',sent1)
print('문장2 :',sent2)

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

문장1 : 캔 윗부분이 볼록하게 찌그러져 왔는데 어떻게 하죠?
문장2 : 문의 주 신상품은 전체 맞교환 진행 예정이며 수거와 출고가 따로 진행되기 때문에 수거기사가 먼저 방문할 수 있는 점 양해 부탁드립니다.


encoder_plus는 정수 인코딩 결과인 input_ids, 두 개의 문장을 구분하기 위한 token_type_ids, 그리고 어텐션 마스크에 해당하는 attention_mask를 자동으로 생성하여 리턴해줍니다. 두 개의 문장을 BERT에 넣을 때는 한 번에 넣어야 하기 때문에 encoder_plus 의 입력으로 sent1과 sent2를 이어서 넣어줍니다.

In [ ]:
encoding_result = tokenizer.encode_plus(sent1, sent2, max_length=max_seq_len,
    pad_to_max_length=True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
print(encoding_result['input_ids'])

[2, 1704, 26468, 2052, 1164, 2351, 2205, 2318, 1625, 14670, 2129, 1458, 13964, 3842, 1889, 2321, 35, 3, 7421, 1564, 18402, 2073, 3910, 1047, 2120, 2018, 3750, 3834, 2052, 2307, 12589, 2522, 23961, 5542, 3750, 2496, 2015, 3624, 2170, 12589, 14036, 2116, 4019, 4110, 2085, 1295, 1513, 2259, 1540, 10797, 5527, 2343, 7316, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
print(encoding_result['token_type_ids'])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
print(encoding_result['attention_mask'])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


어텐션 마스크는 앞에서 풀어왔던 문제들과 마찬가지로 실제 단어 토큰의 위치에는 1이 반복되다가 패딩 토큰인 [PAD]가 등장하는 구간부터는 0이 반복됩니다. 훈련 데이터의 첫번째 샘플을 가지고 임의로 진행했던 전처리를 훈련 데이터, 검증 데이터, 테스트 데이터에 대해서 모두 진행해봅시다. 이 과정은 convert_examples_to_features 함수를 통해서 진행합니다.


In [ ]:
def convert_examples_to_features(sent_list1, sent_list2, max_seq_len, tokenizer):
  input_ids, attention_masks, token_type_ids = [], [], []

  for sent1, sent2 in tqdm(zip(sent_list1, sent_list2), total=len(sent_list1)):
    encoding_result = tokenizer.encode_plus(sent1, sent2, max_length=
        max_seq_len, pad_to_max_length=True)

    input_ids.append(encoding_result['input_ids'])
    attention_masks.append(encoding_result['attention_mask'])
    token_type_ids.append(encoding_result['token_type_ids'])

  input_ids = np.array(input_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)

  return (input_ids, attention_masks, token_type_ids)

In [ ]:
#훈련 데이터에 대해서 전처리를 진행
X_train = convert_examples_to_features(train_data['발화문 (Q)'], train_data['발화문 (A)'],
                                       max_seq_len=max_seq_len, tokenizer=tokenizer)

 35%|███▍      | 44974/130276 [00:35<00:52, 1639.98it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
 36%|███▋      | 47381/130276 [00:36<00:58, 1410.33it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens hav

In [ ]:
input_id = X_train[0][0]
attention_mask = X_train[1][0]
token_type_id = X_train[2][0]

print('단어에 대한 정수 인코딩 :',input_id)
print('어텐션 마스크 :',attention_mask)
print('세그먼트 인코딩 :',token_type_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [    2  1704 26468  2052  1164  2351  2205  2318  1625 14670  2129  1458
 13964  3842  1889  2321    35     3  7421  1564 18402  2073  3910  1047
  2120  2018  3750  3834  2052  2307 12589  2522 23961  5542  3750  2496
  2015  3624  2170 12589 14036  2116  4019  4110  2085  1295  1513  2259
  1540 10797  5527  2343  7316    18     3     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
어텐션 마스크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
#검증 데이터에 대해서 전처리를 진행
X_val = convert_examples_to_features(val_data['발화문 (Q)'], val_data['발화문 (A)'],
     max_seq_len=max_seq_len, tokenizer=tokenizer)

 13%|█▎        | 5592/43429 [00:03<00:21, 1754.81it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
100%|██████████| 43429/43429 [00:31<00:00, 1385.53it/s]


In [ ]:
input_id = X_val[0][0]
attention_mask = X_val[1][0]
token_type_id = X_val[2][0]

print('단어에 대한 정수 인코딩 :',input_id)
print('어텐션 마스크 :',attention_mask)
print('세그먼트 인코딩 :',token_type_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [    2  9488  2052  4252   859  2075  2182    35     3  4867  2205  2250
  3959  2073  3738 15139  2496  2051  9488  1570  6233  3915  3598  3606
    18  4558  2470  4790  2073  1288  2121  2069  3644  9488  6994  3915
  3662 11800    18     3     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
어텐션 마스크 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
#테스트 데이터에 대해서 전처리를 진행
X_test = convert_examples_to_features(test_data['발화문 (Q)'], test_data['발화문 (A)'],
                                      max_seq_len=max_seq_len, tokenizer=tokenizer)

 64%|██████▎   | 27662/43429 [00:20<00:15, 1050.55it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
 93%|█████████▎| 40591/43429 [00:29<00:02, 1211.34it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have 

In [ ]:
#contradiction, entailment, neutral 과 같이 문자열로 구성된 레이블에 대해서도 정수 인코딩을 진행

train_label = train_data['인텐트 (대)'].tolist()
val_label = val_data['인텐트 (대)'].tolist()
test_label = test_data['인텐트 (대)'].tolist()

idx_encode = preprocessing.LabelEncoder()
idx_encode.fit(train_label)

#고유한 정수로 변환
y_train = idx_encode.transform(train_label)
y_val = idx_encode.transform(val_label)
y_test = idx_encode.transform(test_label)

label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))
idx_label = {value: key for key, value in label_idx.items()}

print('각 레이블과 정수 :', label_idx)

각 레이블과 정수 : {'AS': 0, '결제': 1, '교환|반품|환불': 2, '멤버십': 3, '배송': 4, '웹사이트': 5, '제품': 6, '포장': 7, '행사': 8}


In [ ]:
#레이블을 정수 인코딩을 하기 전과 후를 훈련 데이터에 대해서 상위 5개를 출력하여 비교
print('변환 전 :', train_label[:5])
print('변환 후 :', y_train[:5])

변환 전 : ['제품', '배송', '배송', '제품', '배송']
변환 후 : [6 4 4 6 4]


# BERT를 이용한 다중 클래스 분류 모델

BERT 의 CLS 토큰의 출력에 소프트맥스 함수를 사용하는 출력층을 연결합니다. BERT의 출력을 outputs로 리턴받았다면, outputs[1]을 통해 CLS 토큰 위치의 벡터를 사용할 수 있습니다.

In [ ]:
class TFBertForSequenceClassification(tf.keras.Model):
  def __init__(self, model_name, num_labels):
    super(TFBertForSequenceClassification, self).__init__()
    self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
    self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.
                                                    initializers.TruncatedNormal
                                                    (0.02),
                                                activation='softmax',
                                                name='classifier')

  def call(self, inputs):
    input_ids, attention_mask, token_type_ids = inputs
    outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask,
            token_type_ids=token_type_ids)
    cls_token = outputs[1]
    prediction = self.classifier(cls_token)

    return prediction

In [ ]:
# TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [ ]:
with strategy.scope():
  model = TFBertForSequenceClassification("klue/bert-base", num_labels=3)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.SparseCategoricalCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [ ]:
#두 번의 에포크를 수행

early_stopping = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.001,
    patience=2)

model.fit(X_train, y_train, epochs=2, batch_size=32, validation_data = (X_val, y_val),
    callbacks = [early_stopping])

Epoch 1/2
4072/4072 [==============================] - 508s 100ms/step - loss: nan - accuracy: 0.0045 - val_loss: nan - val_accuracy: 0.0040
Epoch 2/2
4072/4072 [==============================] - 355s 87ms/step - loss: nan - accuracy: 0.0045 - val_loss: nan - val_accuracy: 0.0040


In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test, batch_size=1024)[1]))

43/43 [==============================] - 21s 348ms/step - loss: nan - accuracy: 0.0045

 테스트 정확도: 0.0045


# TFBertForSequenceClassification

In [ ]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base",
            num_labels=3, from_pt=True)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
